In [16]:
"""
Created on Mon February 23rd, 2022
@author: Eleftheria Chatzitheodoridou

Description: 
This script is designed to read an excel file that contains tumor boundary information of 
LGG cases from the local directory and check if the segmentator has extracted the tumor area. 
If it failed to do so, it is instructed to extract the (min, max) values from the excel 
file, find those indices on the NIfTI images, extract the 2D slices, and finally, save them
as .png files.
"""

import os.path 
import numpy as np
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from numpy import ndarray

np.seterr(divide = 'ignore', invalid = 'ignore')

# Folder where the created images will be saved in
out_path = r'/local/data1/elech646/Tumor_grade_classification/Slices'

# Create subfolders
if not os.path.exists(out_path + "/sagittal_grade_classification"):
    os.mkdir(out_path + "/sagittal_grade_classification")
    
if not os.path.exists(out_path + "/frontal_grade_classification"):
    os.mkdir(out_path + "/frontal_grade_classification")

if not os.path.exists(out_path + "/trans_grade_classification"):
    os.mkdir(out_path + "/trans_grade_classification")

# Add LGG path
sag_path = out_path + "/sagittal_grade_classification" + "/LGG"
fro_path = out_path + "/frontal_grade_classification" + "/LGG"
tra_path = out_path + "/trans_grade_classification" + "/LGG"

if not os.path.exists(sag_path):
    os.mkdir(sag_path)
    
if not os.path.exists(fro_path):
    os.mkdir(fro_path)
    
if not os.path.exists(tra_path):
    os.mkdir(tra_path)

filepath = []
patient_name = []
# Scanning files and directories
for roots, dirs, files in os.walk("/local/data1/elech646/Tumor_grade_classification/LGG"):
    for name in files:
        if name.endswith((".nii.gz", ".nii")):
            filepath.append(roots + os.path.sep + name)
            patient_name.append(roots.split('/')[-1])

# idx = [i for i, v in enumerate(filepath) if 'TCGA-CS-4944' in v][0]
# print(patient_name[idx])
# print(filepath[idx])

# Get unique patient names
patient_name = list(set(patient_name))
all_paths = []
for x in ['frontal', 'sagittal', 'trans']:
    path_boi = f'/local/data1/elech646/Tumor_grade_classification/Slices/{x}_grade_classification/LGG'
    
    new_filepath = []
    for patient in patient_name:
        folder = path_boi + '/' + patient
        if len(os.listdir(folder)) == 0:
            new_filepath.append(patient)  

    new_filepath = [i for i in filepath if i.split('LGG/')[1].split('/')[0] in new_filepath]
    all_paths.extend(new_filepath)
    
# Import Excel file
annotations_LGG = pd.read_excel(r'/local/data1/elech646/Tumor_grade_classification/LGG case annotations.xlsx')

# Keep only patient id + manual tumor coordinates
annotations_LGG = annotations_LGG[annotations_LGG.Segmented == 'NO'].reset_index(drop = True).\
drop(["CDE_TYPE", "Laterality", "Segmented"], axis = 1)

# Creating the images
for i, file in enumerate(set(all_paths)):
    img = nib.load(file)
    img_data = img.get_fdata()  # Getting data matrix
    patient = file.split("LGG/")[1].split("/")[0]
    
    if not os.path.exists(os.path.join(sag_path, patient)):
        os.mkdir(os.path.join(sag_path, patient))
    if not os.path.exists(os.path.join(fro_path, patient)):
        os.mkdir(os.path.join(fro_path, patient))
    if not os.path.exists(os.path.join(tra_path, patient)):
        os.mkdir(os.path.join(tra_path, patient))
 
    index_patient = [i for i, v in enumerate(annotations_LGG.itertuples()) if v.CDE_ID == patient]
    
    xmin = int(annotations_LGG.at[index_patient[0], 'xmin'])    
    xmax = int(annotations_LGG.at[index_patient[0], 'xmax'])    
    ymin = int(annotations_LGG.at[index_patient[0], 'ymin'])    
    ymax = int(annotations_LGG.at[index_patient[0], 'ymax'])    
    zmin = int(annotations_LGG.at[index_patient[0], 'zmin'])   
    zmax = int(annotations_LGG.at[index_patient[0], 'zmax']) 
    
    # Checking elegantly if the indices are correct
    patient_id = annotations_LGG.CDE_ID.map(str)
    patient_id = patient_id.tolist()
    #print(f'{patient_id[index_patient[0]]} \n xmin: {xmin:3d} xmax: {xmax:3d} \n ymin: {ymin:3d} ymax: {ymax:3d} \n zmin: {zmin:3d} zmax: {zmax:3d}')

    # Creating the images in the Sagittal Plane (yz)
    img_sag = np.rot90(img_data, axes = (1, 2)) # yz plane sagittal
    img_sag = np.flip(img_sag, 0) # flip the image left/right since mango was used
    
    # Checking indices for sagittal case
#     fig, ax = plt.subplots(nrows = 1, ncols = 2)
#     ax[0].imshow(img_sag[xmin,:,:], cmap = 'gray', interpolation = None)
#     ax[0].set_title(f'{patient_id[index_patient[0]]} - Slice {xmin}')
#     ax[1].imshow(img_sag[xmax,:,:], cmap = 'gray', interpolation = None)
#     ax[1].set_title(f'{patient_id[index_patient[0]]} - Slice {xmax}') 
#     plt.show()   

    for sag in range(zmin, zmax + 1):
        perc = int(((sag - zmin)/(zmax - zmin))*100) # Percentage along the selected slices
        tmp = img_sag[sag,:,:]
        min_v = img_sag.min()
        max_v = img_sag.max()
        
        # Normalize image
        tmp_norm = (255*(tmp - min_v) / (max_v - min_v)).astype(np.uint8) 
        
        # Add modality        
        modality = filepath[i].split("_",1)[1].split(".nii.gz", 1)[0].split("_")[-1]
        
        # Name the files
        title = sag_path + "/" + patient + '/' + patient + '_sag_' + modality +\
                '_' + str(sag) + '_' + str(perc) + '.png'
                
        # Convert to RGB
        im = Image.fromarray(tmp_norm).convert('RGB')
        
        # Save images
        im.save(title)
        #print(title)
        
    # Creating the images in the Frontal Plane (xz)
    img_fr = np.rot90(img_data, axes = (0,2)) # xz plane frontal
    
    # Checking indices for frontal case
#     fig, ax = plt.subplots(nrows = 1, ncols = 2)
#     ax[0].imshow(img_fr[:,ymin,:], cmap = 'gray', interpolation = None)
#     ax[0].set_title(f'{patient_id[index_patient[0]]} - Slice {ymin}')
#     ax[1].imshow(img_fr[:,ymax,:], cmap = 'gray', interpolation = None)
#     ax[1].set_title(f'{patient_id[index_patient[0]]} - Slice {ymax}')
#     plt.show()  
    
    for front in range(ymin, ymax + 1):
        perc = int(((front - ymin) /(ymax - ymin))*100) # Percentage along the selected slices
        tmp = img_fr[:,front,:]
        min_v = img_fr.min()
        max_v = img_fr.max()
        
        # Normalize image
        tmp_norm = (255*(tmp - min_v) / (max_v - min_v)).astype(np.uint8) 
        
        # Add modality        
        modality = filepath[i].split("_",1)[1].split(".nii.gz", 1)[0].split("_")[-1]
        
        # Name the files
        title = fro_path + "/" + patient + '/' + patient + '_fro_' + modality +\
                '_' + str(front) + '_' + str(perc) + '.png'
        
        # Convert to RGB
        im = Image.fromarray(tmp_norm).convert('RGB')
        
        # Save images
        im.save(title)
            
    # Creating the images in the Transversal/Axial Plane (xy)
    img_tr = np.rot90(img_data, 3, axes = (0,1)) # xy plane transversal
    x,y,z = img_tr.shape

    # Inverting slices upside/down since mango was used
    xmin = z - zmin
    xmax = z - zmax
    
    # Checking indices for transversal case
#     fig, ax = plt.subplots(nrows = 1, ncols = 2)
#     ax[0].imshow(img_tr[:,:,zmin], cmap = 'gray', interpolation = None)
#     ax[0].set_title(f'{patient_id[index_patient[0]]} - Slice {zmin}')
#     ax[1].imshow(img_tr[:,:,zmax], cmap = 'gray', interpolation = None)
#     ax[1].set_title(f'{patient_id[index_patient[0]]} - Slice {zmax}')
#     plt.show()
    
    for transv in range(xmin, xmax, -1):
        perc = int(((transv - xmin) / (xmax - xmin))*100) # Percentage along the selected slices
        tmp = img_tr[:,:,transv]
        min_v = img_tr.min()
        max_v = img_tr.max()

        # Normalize image
        tmp_norm = (255*(tmp - min_v) / (max_v - min_v)).astype(np.uint8)
        
        # Add modality        
        modality = filepath[i].split("_",1)[1].split(".nii.gz", 1)[0].split("_")[-1]
        
        # Name the files
        title = tra_path + "/" + patient + '/' + patient + '_trans_' + modality +\
                '_' + str(transv) + '_' + str(perc) + '.png'
        
        # Convert to RGB
        im = Image.fromarray(tmp_norm).convert('RGB')
 
        # Save images
        im.save(title)